# Machine Learning Language Translation

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [13]:
batch_size=64 # batch size of traning
epochs=100    #Number of epoch to train for
latent_dim = 256 # Latent dimentionality of the encoding space
num_samples =1000 #Number of sample to train on
df = 'fra.txt'

In [16]:
#Vectorize the data
input_texts =[]
target_texts =[]
input_characters =set()
target_characters =set()
with open(df, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
    input_text,target_text,_ = line.split('\t')
    #we use 'tab' as 'start sequence' character
    #for the targets, and "\n" as "end sequence" character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [17]:
input_characters =sorted(list(input_characters))
target_characters =sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [18]:
print('Number of sample:',len(input_texts))
print('Number of unique input tokens:',num_encoder_tokens)
print('Number of unique output tokens:',num_decoder_tokens)
print('Max sequence length for inputs:',max_encoder_seq_length)
print('Max sequence length for outputs:',max_decoder_seq_length)

Number of sample: 1000
Number of unique input tokens: 58
Number of unique output tokens: 74
Max sequence length for inputs: 10
Max sequence length for outputs: 33


In [19]:
input_token_index = dict(
        [(char,i) for i,char in enumerate(input_characters)])
target_token_index = dict(
        [(char,i) for i,char in enumerate(target_characters)])

In [20]:
input_token_index ,target_token_index 

({' ': 0,
  '!': 1,
  '$': 2,
  "'": 3,
  ',': 4,
  '.': 5,
  '1': 6,
  '5': 7,
  '9': 8,
  '?': 9,
  'A': 10,
  'B': 11,
  'C': 12,
  'D': 13,
  'E': 14,
  'F': 15,
  'G': 16,
  'H': 17,
  'I': 18,
  'J': 19,
  'K': 20,
  'L': 21,
  'M': 22,
  'N': 23,
  'O': 24,
  'P': 25,
  'R': 26,
  'S': 27,
  'T': 28,
  'U': 29,
  'W': 30,
  'Y': 31,
  'a': 32,
  'b': 33,
  'c': 34,
  'd': 35,
  'e': 36,
  'f': 37,
  'g': 38,
  'h': 39,
  'i': 40,
  'j': 41,
  'k': 42,
  'l': 43,
  'm': 44,
  'n': 45,
  'o': 46,
  'p': 47,
  'q': 48,
  'r': 49,
  's': 50,
  't': 51,
  'u': 52,
  'v': 53,
  'w': 54,
  'x': 55,
  'y': 56,
  'z': 57},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  "'": 4,
  ',': 5,
  '-': 6,
  '.': 7,
  '1': 8,
  '9': 9,
  '?': 10,
  'A': 11,
  'B': 12,
  'C': 13,
  'D': 14,
  'E': 15,
  'F': 16,
  'G': 17,
  'H': 18,
  'I': 19,
  'J': 20,
  'L': 21,
  'M': 22,
  'N': 23,
  'O': 24,
  'P': 25,
  'Q': 26,
  'R': 27,
  'S': 28,
  'T': 29,
  'U': 30,
  'V': 31,
  'a': 32,
  'b': 33,
  'c'

In [21]:
encoder_input_data=np.zeros(
      (len(input_texts), max_encoder_seq_length,num_encoder_tokens),
     dtype='float32')
decoder_input_data=np.zeros(
      (len(input_texts), max_decoder_seq_length,num_decoder_tokens),
     dtype='float32')
decoder_target_data=np.zeros(
      (len(input_texts), max_decoder_seq_length,num_decoder_tokens),
     dtype='float32')

In [22]:
for i, (input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        #decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t>0:
            #decoder_target_data will be ahead by one timestep
            #and will not include the start character.
            decoder_target_data[i, t-1, target_token_index[char]] =1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [24]:
encoder_input_data[0].shape

(10, 58)

In [26]:
#Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim,return_state=True)
encoder_outputs, state_h,state_c = encoder(encoder_inputs)
#we discard encoder outputs and only keep the states
encoder_states = [state_h, state_c]

In [27]:
#set up decoder,using "encoder_states" as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))
#We set up our decoder to return full output sequences, and to return internal states as well.
# We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens,activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [28]:
#Define the model that will turn 'encoder_input_data & decoder_input_data into' 'decoder_target_data' 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Run training
model.compile(optimizer='rmsprop', loss ='categorical_crossentropy', metrics = ['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_split=0.2)

Epoch 1/100
13/13 [==============================] - 16s 408ms/step - loss: 2.3737 - accuracy: 0.5824 - val_loss: 1.9218 - val_accuracy: 0.5961
Epoch 2/100
13/13 [==============================] - 3s 241ms/step - loss: 1.6702 - accuracy: 0.6339 - val_loss: 1.7896 - val_accuracy: 0.5961
Epoch 3/100
13/13 [==============================] - 3s 239ms/step - loss: 1.5884 - accuracy: 0.6350 - val_loss: 1.8300 - val_accuracy: 0.6000
Epoch 4/100
13/13 [==============================] - 3s 243ms/step - loss: 1.5659 - accuracy: 0.6380 - val_loss: 1.6635 - val_accuracy: 0.6044
Epoch 5/100
13/13 [==============================] - 3s 240ms/step - loss: 1.5093 - accuracy: 0.6390 - val_loss: 1.7287 - val_accuracy: 0.6052
Epoch 6/100
13/13 [==============================] - 3s 242ms/step - loss: 1.4810 - accuracy: 0.6394 - val_loss: 1.7167 - val_accuracy: 0.5945
Epoch 7/100
13/13 [==============================] - 3s 241ms/step - loss: 1.4415 - accuracy: 0.6402 - val_loss: 1.6416 - val_accuracy: 0.603

Epoch 58/100
13/13 [==============================] - 3s 242ms/step - loss: 0.6866 - accuracy: 0.7995 - val_loss: 0.9925 - val_accuracy: 0.7171
Epoch 59/100
13/13 [==============================] - 3s 237ms/step - loss: 0.6840 - accuracy: 0.8009 - val_loss: 0.9991 - val_accuracy: 0.7148
Epoch 60/100
13/13 [==============================] - 3s 234ms/step - loss: 0.6706 - accuracy: 0.8042 - val_loss: 0.9701 - val_accuracy: 0.7238
Epoch 61/100
13/13 [==============================] - 3s 234ms/step - loss: 0.6711 - accuracy: 0.8045 - val_loss: 1.0092 - val_accuracy: 0.7064
Epoch 62/100
13/13 [==============================] - 3s 235ms/step - loss: 0.6646 - accuracy: 0.8041 - val_loss: 0.9478 - val_accuracy: 0.7333
Epoch 63/100
13/13 [==============================] - 3s 243ms/step - loss: 0.6629 - accuracy: 0.8066 - val_loss: 0.9624 - val_accuracy: 0.7232
Epoch 64/100
13/13 [==============================] - 3s 242ms/step - loss: 0.6519 - accuracy: 0.8078 - val_loss: 0.9516 - val_accuracy:

In [33]:
#Next: inference mode(sampling)
#Here's the drill:
# 1) encode input and retrive initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs,encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs =[decoder_state_input_h,decoder_state_input_c]
decoder_outputs, state_h,state_c = decoder_lstm(
           decoder_inputs,initial_state=decoder_states_inputs)
decoder_states = [state_h,state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
        [decoder_inputs] +decoder_states_inputs,
         [decoder_outputs] + decoder_states) 
#Reverse lookup token index to decode sequences back to something readable 
reverse_input_char_index = dict(
         (i,char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
         (i,char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    #encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    #Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1, num_decoder_tokens))
    #populate the first character of the target sequence with the start character.
    target_seq[0,0, target_token_index['\t']] = 1.
    
    #sampling loop for a batch sequences
    #(to simplify here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ' '
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        #sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        #Exit condation: either hit max length
        #or find stop character.
        if (sampled_char =='\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
            
        #update the target sequence of length 1    
        target_seq = np.zeros((1,1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index] = 1.
        
        #update states
        states_value = [h,c] 
        
    return decoded_sentence

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index:seq_index +1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Go.
Decoded sentence:  Allez !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Go.
Decoded sentence:  Allez !

1/1 [==============================] - 0s 45ms/step
-
Input sentence: Go.
Decoded sentence:  Allez !

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Hi.
Decoded sentence:  Couss ju mer !

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Hi.
Decoded sentence:  Couss ju mer !

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Run!
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Run!
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 44ms/step
-
Input sentence: Run!
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 49ms/step
-
Input sentence: Run!
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 50ms/step
-
Input sent

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Run.
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 49ms/step
-
Input sentence: Run.
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 39ms/step
-
Input sentence: Run.
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 45ms/step
-
Input sentence: Run.
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 64ms/step
-
Input sentence: Run.
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 41ms/step
-
Input sentence: Run.
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Run.
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Run.
Decoded sentence:  Fuyez !

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Who?
Decoded sentence:  Tom e gagné.

1/1 [==============================] - 0s 47ms/step
-
Input sentence

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Duck!
Decoded sentence:  Faisse temben !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Fire!
Decoded sentence:  Aussez vous.

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Help!
Decoded sentence:  Fient !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Hide.
Decoded sentence:  Enton !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Hide.
Decoded sentence:  Enton !

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Jump!
Decoded sentence:  Sourez !

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Jump.
Decoded sentence:  Souss !

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Stop!
Decoded sentence:  Attendez.

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Stop!
Decoded sentence:  Attendez.

1/1 [==============================] - 0s 52ms

1/1 [==============================] - 0s 47ms/step
-
Input sentence: Wait.
Decoded sentence:  Attends couc.

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Wait.
Decoded sentence:  Attends couc.

1/1 [==============================] - 0s 41ms/step
-
Input sentence: Wait.
Decoded sentence:  Attends couc.

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Wait.
Decoded sentence:  Attends couc.

1/1 [==============================] - 0s 49ms/step
-
Input sentence: Begin.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 52ms/step
-
Input sentence: Begin.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Go on.
Decoded sentence:  Allez !

1/1 [==============================] - 0s 65ms/step
-
Input sentence: Go on.
Decoded sentence:  Allez !

1/1 [==============================] - 0s 47ms/step
-
Input sentence: Go on.
Decoded sentence:  Allez !

1/1 [==================

1/1 [==============================] - 0s 40ms/step
-
Input sentence: I see.
Decoded sentence:  J'ai faité.

1/1 [==============================] - 0s 48ms/step
-
Input sentence: I see.
Decoded sentence:  J'ai faité.

1/1 [==============================] - 0s 65ms/step
-
Input sentence: I try.
Decoded sentence:  J'ai faité.

1/1 [==============================] - 0s 47ms/step
-
Input sentence: I won!
Decoded sentence:  J'ai faité.

1/1 [==============================] - 0s 40ms/step
-
Input sentence: I won!
Decoded sentence:  J'ai faité.

1/1 [==============================] - 0s 48ms/step
-
Input sentence: I won.
Decoded sentence:  J'ai faité.

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Oh no!
Decoded sentence:  Couss gagne !

1/1 [==============================] - 0s 42ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [========

1/1 [==============================] - 0s 49ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 41ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Relax.
Decoded sentence:  Soyez campe !

1/1 [==============================] - 0s 47ms/step
-
Input sentence: Smile.
Decoded sentence:  Sourez-moi !

1/

1/1 [==============================] - 0s 47ms/step
-
Input sentence: Smile.
Decoded sentence:  Sourez-moi !

1/1 [==============================] - 0s 49ms/step
-
Input sentence: Sorry?
Decoded sentence:  Fais !

1/1 [==============================] - 0s 45ms/step
-
Input sentence: Attack!
Decoded sentence:  Attendez.

1/1 [==============================] - 0s 35ms/step
-
Input sentence: Attack!
Decoded sentence:  Attendez.

1/1 [==============================] - 0s 39ms/step
-
Input sentence: Attack!
Decoded sentence:  Attendez.

1/1 [==============================] - 0s 31ms/step
-
Input sentence: Buy it.
Decoded sentence:  Attendez-le.

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Buy it.
Decoded sentence:  Attendez-le.

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Buy it.
Decoded sentence:  Attendez-le.

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Buy it.
Decoded sentence:  Attendez-le.

1/1 [=============

1/1 [==============================] - 0s 65ms/step
-
Input sentence: Cheers!
Decoded sentence:  Entondez à Tom.

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Cheers!
Decoded sentence:  Entondez à Tom.

1/1 [==============================] - 0s 41ms/step
-
Input sentence: Eat it.
Decoded sentence:  Attendez-le.

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Eat it.
Decoded sentence:  Attendez-le.

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Get up.
Decoded sentence:  Dégage !

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Get up.
Decoded sentence:  Dégage !

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Get up.
Decoded sentence:  Dégage !

1/1 [==============================] - 0s 41ms/step
-
Input sentence: Go now.
Decoded sentence:  Allez !

1/1 [==============================] - 0s 55ms/step
-
Input sentence: Go now.
Decoded sentence:  Allez !

1/1 [============

1/1 [==============================] - 0s 32ms/step
-
Input sentence: Got it!
Decoded sentence:  Dégage !

1/1 [==============================] - 0s 47ms/step
-
Input sentence: Got it?
Decoded sentence:  Attendez.

1/1 [==============================] - 0s 50ms/step
-
Input sentence: Got it?
Decoded sentence:  Attendez.

1/1 [==============================] - 0s 33ms/step
-
Input sentence: Got it?
Decoded sentence:  Attendez.

1/1 [==============================] - 0s 39ms/step
-
Input sentence: Hop in.
Decoded sentence:  Allez !

1/1 [==============================] - 0s 48ms/step
-
Input sentence: Hop in.
Decoded sentence:  Allez !

1/1 [==============================] - 0s 40ms/step
-
Input sentence: Hug me.
Decoded sentence:  Soyez camme !

